## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from tqdm import tqdm

## Load train and test data

In [2]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

## Classifiers & Features

In [3]:
classifiers = {'knn': KNeighborsClassifier(), 'nb': MultinomialNB(), 'sgdc': SGDClassifier()}
features = {'counts': None, 'tf': TfidfTransformer(use_idf=False), 'tfidf': TfidfTransformer()}

## Sets of parameters for Grid Search

In [4]:
parameters = [
            {
                'vect__lowercase': (True, False),
            },
            {
                'vect__stop_words': (None, 'english'),
            },
            {
                'vect__analyzer': ['word'],
                'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
            },
            {
                'vect__analyzer': ['char'],
                'vect__ngram_range': [(2, 2), (3, 3), (4, 4)]
            },
            {
                'vect__max_features': (1000, 5000, 10000)
            }
]

In [5]:
doc_root_knn = int(np.sqrt(len(twenty_train.data)))
half_root_knn = int(doc_root_knn / 2)
hyperparameters = {
                  "knn": {
                              "clf__n_neighbors": [5, 11, 21, half_root_knn, doc_root_knn]
                        },
                  "nb": {
                              "clf__alpha": [0.0001, 0.001, 0.01, 0.1, 1]
                        },
                  "sgdc": {
                              "clf__alpha": [0.0001, 0.001, 0.01, 0.1, 1]
                          }
                  }

## Combinations of pipelines

In [6]:
def build_pipelines(classifiers: dict, features: dict):
    pipelines = {}
    for clf_key in classifiers.keys():
        for feat_key in features.keys():
            clf = classifiers.get(clf_key)
            feat = features.get(feat_key)
            pipe = Pipeline([
                            ('vect', CountVectorizer()),
                            ('tfidf', feat),
                            ('clf', clf)
            ])
            pipelines[(clf_key, feat_key)] = pipe
    return pipelines

In [7]:
pipelines = build_pipelines(classifiers=classifiers, features=features)

## First Experiment
Test with all the default settings, each classifier with each of the parameters. No parameters

In [8]:
all_mean_acc = []
all_prf = []
all_conf = []
classifiers = []
features = []

for key, pipe in tqdm(pipelines.items()):
    pipe.fit(twenty_train.data, twenty_train.target)
    predicted = pipe.predict(twenty_test.data)

    # mean accuracy
    mean_acc = np.mean(predicted == twenty_test.target)
    all_mean_acc.append(mean_acc)

    # precision, recall, f1 score
    prf = metrics.precision_recall_fscore_support(twenty_test.target, predicted, average='weighted')
    all_prf.append(prf)

    conf_matrix = metrics.confusion_matrix(twenty_test.target, predicted)
    all_conf.append(conf_matrix)

    classifiers.append(key[0])
    features.append(key[1])

df_data = {"Classifier": classifiers,
            "Features": features,
            "Mean Accuracy": all_mean_acc
          }
results = pd.DataFrame(df_data)
prf_df = pd.DataFrame(all_prf, columns=["Precision", "Recall", "F1", "_"]).drop(columns=['_'])
results = results.merge(prf_df, left_index=True, right_index=True)
results = results.round(3)
results.to_csv('results/experiment_default_knn.csv', index=False)
results


100%|██████████| 9/9 [00:48<00:00,  5.39s/it]


,Classifier,Features,Mean Accuracy,Precision,Recall,F1
0,knn,counts,0.352,0.421,0.352,0.357
1,knn,tf,0.408,0.489,0.408,0.417
2,knn,tfidf,0.659,0.674,0.659,0.660
3,nb,counts,0.773,0.762,0.773,0.751
4,nb,tf,0.705,0.785,0.705,0.692
5,nb,tfidf,0.774,0.822,0.774,0.768
6,sgdc,counts,0.758,0.784,0.758,0.759
7,sgdc,tf,0.808,0.813,0.808,0.804
8,sgdc,tfidf,0.852,0.853,0.852,0.850


## Hyperparameter Tuning on Pipelines

In [9]:
all_mean_acc = []
all_prf_hyper = []
best_pipelines = {}
best_params = []
classifier_names = []
feature_names = []
all_cv_results = []
for key, pipe in tqdm(pipelines.items()):
    parameter_set = hyperparameters[key[0]]

    gs_clf = GridSearchCV(pipe, parameter_set, cv=5, n_jobs=-1)
    gs_clf.fit(twenty_train.data, twenty_train.target)
    predicted = gs_clf.predict(twenty_test.data)

    all_cv_results.append(gs_clf.cv_results_)
    
    mean_acc = np.mean(predicted == twenty_test.target)
    all_mean_acc.append(mean_acc)
    prf = metrics.precision_recall_fscore_support(twenty_test.target, predicted, average='weighted')
    all_prf_hyper.append(prf)

    best_estimator = gs_clf.best_estimator_
    best_pipelines[key] = best_estimator
    # best_pipelines.append(best_estimator)

    best_param = gs_clf.best_params_
    best_params.append(best_param)

    classifier_names.append(key[0])
    feature_names.append(key[1])
    
df_data = {"Classifier": classifier_names,
    "Features": feature_names,
    "Best Parameters": best_params,
    "Mean Accuracy": all_mean_acc
    }
results_exp2 = pd.DataFrame(df_data)
prf_hyper_df = pd.DataFrame(all_prf_hyper, columns=["Precision", "Recall", "F1", "_"]).drop(columns=['_'])
results_exp2 = results_exp2.merge(prf_hyper_df, left_index=True, right_index=True)
results_exp2 = results_exp2.round(3)
results_exp2.to_csv('results/experiment2_knn.csv')
results_exp2

100%|██████████| 9/9 [02:39<00:00, 17.73s/it]


,Classifier,Features,Best Parameters,Mean Accuracy,Precision,Recall,F1
0,knn,counts,{'clf__n_neighbors': 5},0.352,0.421,0.352,0.357
1,knn,tf,{'clf__n_neighbors': 5},0.408,0.489,0.408,0.417
2,knn,tfidf,{'clf__n_neighbors': 5},0.659,0.674,0.659,0.660
3,nb,counts,{'clf__alpha': 0.0001},0.797,0.804,0.797,0.785
4,nb,tf,{'clf__alpha': 0.001},0.833,0.834,0.833,0.831
5,nb,tfidf,{'clf__alpha': 0.01},0.835,0.836,0.835,0.834
6,sgdc,counts,{'clf__alpha': 0.01},0.794,0.803,0.794,0.788
7,sgdc,tf,{'clf__alpha': 0.0001},0.811,0.811,0.811,0.806
8,sgdc,tfidf,{'clf__alpha': 0.0001},0.853,0.854,0.853,0.852


## Third Experiment: Vectorizer Parameters
Uses best estimators from previous experiment

In [10]:
prf_params = []
mean_acc_params = []
classifier_names = []
feature_names = []
vect_params = []

all_cv_results_params = []

for key, pipe in tqdm(best_pipelines.items()):
    for params in parameters:
        gs_clf = GridSearchCV(pipe, params, cv=5, n_jobs=-1)
        gs_clf.fit(twenty_train.data, twenty_train.target)
        predicted = gs_clf.predict(twenty_test.data)

        mean_acc = np.mean(predicted == twenty_test.target)
        mean_acc_params.append(mean_acc)
        prf = metrics.precision_recall_fscore_support(twenty_test.target, predicted, average='weighted')
        prf_params.append(prf)

        vect_params.append(gs_clf.best_params_)
        best_estimator = gs_clf.best_estimator_
        classifier_names.append(key[0])
        feature_names.append(key[1])

        all_cv_results_params.append(gs_clf.cv_results_)

df_data = {"Classifier": classifier_names,
            "Features": feature_names,
            "Vectorizer Parameters": vect_params,
            "Mean Accuracy": mean_acc_params
          }
results_exp3 = pd.DataFrame(df_data)
prf_params_df = pd.DataFrame(prf_params, columns=["Precision", "Recall", "F1", "_"]).drop(columns=['_'])
results_exp3 = results_exp3.merge(prf_params_df, left_index=True, right_index=True)
results_exp3 = results_exp3.round(3)
results_exp3.to_csv('results/experiment3_knn.csv')
results_exp3

 22%|██▏       | 2/9 [05:16<18:27, 158.22s/it]/Users/andreassavva/opt/anaconda3/envs/ml/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
 89%|████████▉ | 8/9 [16:53<02:00, 120.32s/it]/Users/andreassavva/opt/anaconda3/envs/ml/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py:702: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
100%|██████████| 9/9 [18:59<00:00, 126.61s/it]


,Classifier,Features,Vectorizer Parameters,Mean Accuracy,Precision,Recall,F1
0,knn,counts,{'vect__lowercase': True},0.352,0.421,0.352,0.357
1,knn,counts,{'vect__stop_words': 'english'},0.364,0.567,0.364,0.401
2,knn,counts,"{'vect__analyzer': 'word', 'vect__ngram_range'...",0.352,0.421,0.352,0.357
3,knn,counts,"{'vect__analyzer': 'char', 'vect__ngram_range'...",0.394,0.435,0.394,0.399
4,knn,counts,{'vect__max_features': 10000},0.353,0.408,0.353,0.361
5,knn,tf,{'vect__lowercase': True},0.408,0.489,0.408,0.417
6,knn,tf,{'vect__stop_words': 'english'},0.624,0.637,0.624,0.626
7,knn,tf,"{'vect__analyzer': 'word', 'vect__ngram_range'...",0.514,0.553,0.514,0.519
8,knn,tf,"{'vect__analyzer': 'char', 'vect__ngram_range'...",0.442,0.513,0.442,0.453
9,knn,tf,{'vect__max_features': 10000},0.413,0.478,0.413,0.421
